## 0. 실행 전 체크

### 필요 사항
- PyRosetta 설치/라이선스가 현재 커널에서 정상 동작해야 함
- 입력 파일이 노트북 작업 폴더에 존재해야 함  
  - 예: `fold_test1_model_0.pdb` (AF3 결과 PDB)

### 권장 실행 순서
위에서 아래로 순서대로 셀 실행

# 입력: AlphaFold3 결과 CIF만 업로드해서 시작

- 업로드 파일: `fold_test1_model_0.cif`
- 1단계: BioPython으로 CIF → PDB 변환
- 2단계: 변환된 PDB를 PyRosetta 파이프라인 입력으로 사용

In [2]:
import os
from pathlib import Path

INPUT_CIF = "../data/fold_test1/fold_test1_model_0.cif"
OUTPUT_PDB = "../data/fold_test1/fold_test1_model_0_from_cif.pdb"

print("INPUT_CIF:", INPUT_CIF)
assert os.path.exists(INPUT_CIF), f"CIF 파일이 없습니다: {INPUT_CIF}"
print("[OK] CIF exists")

INPUT_CIF: ../data/fold_test1/fold_test1_model_0.cif
[OK] CIF exists


In [3]:
%pwd

'/mnt/g/repos/bio/notebooks'

In [4]:
# BioPython CIF -> PDB 변환
# 필요 패키지: biopython (conda: conda install -c conda-forge biopython)

from Bio.PDB import MMCIFParser, PDBIO

def cif_to_pdb(cif_path: str, pdb_path: str, structure_id="AF3_MODEL"):
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure(structure_id, cif_path)

    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_path)

    return pdb_path

out = cif_to_pdb(INPUT_CIF, OUTPUT_PDB)
print("[OK] Converted CIF -> PDB:", out)
print("File size (bytes):", os.path.getsize(out))

[OK] Converted CIF -> PDB: ../data/fold_test1/fold_test1_model_0_from_cif.pdb
File size (bytes): 241691


In [5]:
INPUT_PDB = OUTPUT_PDB
print("[OK] Pipeline input PDB set to:", INPUT_PDB)

[OK] Pipeline input PDB set to: ../data/fold_test1/fold_test1_model_0_from_cif.pdb


# SSTR2–SST14 설계 데모 (Notebook)

## 목표
이 노트북은 AlphaFold3 결과로 얻은 SSTR2–SST14 복합체 구조를 기반으로:

1) 입력 PDB 로드 및 체인/서열 요약을 텍스트로 확인  
2) **펩타이드(길이 14) 체인 자동 탐지**  
3) **체인 표준화: A=리셉터, B=펩타이드** (A/B 하드코딩 위험 제거)  
4) **펩타이드만 Relax** (리셉터는 고정)  
5) **FastDesign 후보 20개 생성 + 스코어링(dG/dSASA + 안정성 proxy)**  
6) **FlexPepDock으로 후보 refine + 재스코어링**

## 데모에서 강조할 아키텍처 포인트
- **체인 표준화(Chain standardization)** 를 통해 Interface 분석 지표가 흔들리지 않게 만듭니다.
- FastDesign은 “설계 후보 생성”, FlexPepDock은 “정밀 검증(refine)” 역할로 분리합니다.
- 혈중 안정성(6~10일) 자체를 정확히 예측하진 않더라도, **stability/PK proxy 레이어**를 넣어 멀티 오브젝티브 선별 구조를 데모로 보여줍니다.

In [6]:
# 1 기본 라이브러리
import os
import csv
import time
from collections import defaultdict, OrderedDict

# 2 표/정렬 출력용
import pandas as pd
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.width", 140)

# 3 진행 상태 표시
from tqdm.notebook import tqdm

# 4 IPython 표시 유틸
from IPython.display import display, HTML

# 5 인터랙티브 3D 분자 뷰어
import py3Dmol

print("Libraries imported (incl. tqdm + py3Dmol for 3D visualization)")

Libraries imported (incl. tqdm + py3Dmol for 3D visualization)


In [7]:
# =========================================================
# py3Dmol 헬퍼 함수 (노트북 전체에서 재사용)
# =========================================================

def show_structure_3d(pdb_path, width=800, height=500,
                      receptor_color="lightblue", peptide_color="orange",
                      surface_receptor=False, stick_peptide=True,
                      label=None):
    """PDB 파일을 py3Dmol 인터랙티브 3D 뷰어로 표시.
    
    Args:
        pdb_path: PDB 파일 경로
        width, height: 뷰어 크기 (px)
        receptor_color: 체인 A(리셉터) 색상
        peptide_color: 체인 B(펩타이드) 색상
        surface_receptor: 리셉터 표면 렌더링 여부
        stick_peptide: 펩타이드 stick 스타일 추가 여부
        label: 뷰어 위에 표시할 라벨 텍스트
    """
    with open(pdb_path, "r") as f:
        pdb_data = f.read()
    
    view = py3Dmol.view(width=width, height=height)
    view.addModel(pdb_data, "pdb")
    
    # 리셉터(Chain A) - cartoon
    view.setStyle({"chain": "A"}, {"cartoon": {"color": receptor_color, "opacity": 0.85}})
    
    # 펩타이드(Chain B) - cartoon + stick
    view.setStyle({"chain": "B"}, {"cartoon": {"color": peptide_color}})
    if stick_peptide:
        view.addStyle({"chain": "B"}, {"stick": {"colorscheme": "orangeCarbon", "radius": 0.15}})
    
    # 리셉터 표면 (옵션)
    if surface_receptor:
        view.addSurface(py3Dmol.VDW, {"opacity": 0.3, "color": receptor_color}, {"chain": "A"})
    
    # 라벨
    if label:
        view.addLabel(label, {"backgroundColor": "white", "fontColor": "black",
                              "fontSize": 14, "position": {"x": 0, "y": 0, "z": 0}})
    
    view.zoomTo()
    view.setBackgroundColor("white")
    return view


def show_comparison_3d(pdb_paths, labels=None, width=800, height=500,
                       receptor_color="lightblue", peptide_colors=None):
    """여러 PDB 구조를 나란히 그리드로 비교 표시.
    
    Args:
        pdb_paths: PDB 파일 경로 리스트
        labels: 각 구조의 라벨 리스트
        width, height: 전체 뷰어 크기 (px)
        receptor_color: 리셉터 색상
        peptide_colors: 펩타이드 색상 리스트
    """
    n = len(pdb_paths)
    if labels is None:
        labels = [f"#{i+1}" for i in range(n)]
    if peptide_colors is None:
        _palette = ["orange", "hotpink", "lime", "cyan", "yellow", "red", "magenta"]
        peptide_colors = [_palette[i % len(_palette)] for i in range(n)]
    
    # 그리드 레이아웃: 최대 3열
    cols = min(n, 3)
    rows = (n + cols - 1) // cols
    
    view = py3Dmol.view(width=width, height=height * rows // max(rows, 1),
                        viewergrid=(rows, cols), linked=False)
    
    for idx, pdb_path in enumerate(pdb_paths):
        r, c = divmod(idx, cols)
        with open(pdb_path, "r") as f:
            pdb_data = f.read()
        
        view.addModel(pdb_data, "pdb", viewer=(r, c))
        view.setStyle({"chain": "A"}, {"cartoon": {"color": receptor_color, "opacity": 0.8}}, viewer=(r, c))
        view.setStyle({"chain": "B"}, {"cartoon": {"color": peptide_colors[idx]}}, viewer=(r, c))
        view.addStyle({"chain": "B"}, {"stick": {"colorscheme": "orangeCarbon", "radius": 0.12}}, viewer=(r, c))
        
        # 각 뷰어에 라벨
        view.addLabel(labels[idx],
                      {"backgroundColor": "white", "fontColor": "black", "fontSize": 12,
                       "position": {"x": -20, "y": 20, "z": 0}, "backgroundOpacity": 0.8},
                      viewer=(r, c))
        view.zoomTo(viewer=(r, c))
    
    view.setBackgroundColor("white")
    return view


print("✅ 3D 뷰어 헬퍼 함수 정의 완료: show_structure_3d(), show_comparison_3d()")

✅ 3D 뷰어 헬퍼 함수 정의 완료: show_structure_3d(), show_comparison_3d()


In [8]:
# PyRosetta 로드
# 주의: 커널에서 init은 보통 1회만 수행하는 것이 안전합니다.
import pyrosetta
from pyrosetta import rosetta

pyrosetta.init("-mute all -relax:default_repeats 3")
print("PyRosetta initialized")

┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2026 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python312.Release 2026.06+release.1a56185c2592611dec4c9c75ddc9468cd2227c1f 2026-01-30T13:14:27] retrieved from: http://www.pyrosetta.org
PyRosetta initialized


# 1. 입력 파일 지정

여기서는 AF3 결과 PDB(`fold_test1_model_0.pdb`)를 입력으로 사용합니다.

- 입력 PDB에는 체인 A/B가 입력 순서에 따라 뒤바뀔 수 있으므로
- **표준화 단계에서 A=리셉터, B=펩타이드로 강제**합니다.

In [9]:
# 입력 PDB 파일명 (CIF→PDB 변환 결과)
INPUT_PDB = "../data/fold_test1/fold_test1_model_0_from_cif.pdb"

print("INPUT_PDB:", INPUT_PDB)
assert os.path.exists(INPUT_PDB), f"파일이 없습니다: {INPUT_PDB}"

INPUT_PDB: ../data/fold_test1/fold_test1_model_0_from_cif.pdb


# 2. PDB 텍스트 기반 체인 요약(사전 점검)

PyRosetta로 들어가기 전에, PDB 파일 자체(ATOM 라인 기반)에서
- 체인 ID
- 잔기 수(길이)
- residue 번호 범위
- 서열 일부
를 빠르게 확인합니다.

In [10]:
# 3-letter -> 1-letter 매핑 (표준 아미노산 중심)
AA3_TO_1 = {
    "ALA": "A", "ARG": "R", "ASN": "N", "ASP": "D", "CYS": "C",
    "GLN": "Q", "GLU": "E", "GLY": "G", "HIS": "H", "ILE": "I",
    "LEU": "L", "LYS": "K", "MET": "M", "PHE": "F", "PRO": "P",
    "SER": "S", "THR": "T", "TRP": "W", "TYR": "Y", "VAL": "V",
    "MSE": "M",
}

def parse_pdb_residues(pdb_path: str):
    """PDB를 ATOM/HETATM 라인 기준으로 파싱하여 chain->residue list를 만듦"""
    chains = defaultdict(OrderedDict)
    with open(pdb_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if not (line.startswith("ATOM") or line.startswith("HETATM")):
                continue
            if len(line) < 27:
                continue
            chain_id = line[21].strip() or "?"
            resname = line[17:20].strip().upper()
            resseq_raw = line[22:26].strip()
            icode = line[26].strip()
            try:
                resseq = int(resseq_raw)
            except ValueError:
                continue
            key = (resseq, icode)
            if key not in chains[chain_id]:
                chains[chain_id][key] = resname
    return chains

def residues_to_seq(res_dict: OrderedDict):
    return "".join(AA3_TO_1.get(res3, "X") for res3 in res_dict.values())

def contiguous_ranges(res_keys):
    nums = [k[0] for k in res_keys]
    if not nums:
        return []
    ranges = []
    start = prev = nums[0]
    for n in nums[1:]:
        if n == prev + 1:
            prev = n
        else:
            ranges.append((start, prev))
            start = prev = n
    ranges.append((start, prev))
    return [f"{a}-{b}" if a != b else f"{a}" for a, b in ranges]

def summarize_pdb(pdb_path, show_seq="head"):
    chains = parse_pdb_residues(pdb_path)
    rows = []
    for cid, res_dict in chains.items():
        keys = list(res_dict.keys())
        seq = residues_to_seq(res_dict)
        if show_seq == "head":
            seq_out = seq[:60] + ("..." if len(seq) > 60 else "")
        elif show_seq == "full":
            seq_out = seq
        else:
            seq_out = ""
        rows.append({
            "chain": cid,
            "length": len(keys),
            "pdb_res_min": keys[0][0] if keys else None,
            "pdb_res_max": keys[-1][0] if keys else None,
            "ranges": ", ".join(contiguous_ranges(keys)),
            "seq": seq_out,
        })
    return pd.DataFrame(rows).sort_values(["chain"]).reset_index(drop=True)

df_input_summary = summarize_pdb(INPUT_PDB, show_seq="head")
df_input_summary

,chain,length,pdb_res_min,pdb_res_max,ranges,seq
0,A,14,1,14,1-14,AGCKNFFWKTFTSC
1,B,369,1,369,1-369,MDMADEPLNGSHTWLSIPFDLNGSVVSTNTSNQTEPYYDLTSNAVLTFIYFVVCIIGLCG...


# 3. PyRosetta Pose 로딩 + 펩타이드 체인 탐지(길이==14)

데모 재현성을 위해 펩타이드를 **길이==14**로 강하게 탐지합니다.

- 히트가 1개가 아니면 오류 처리(입력 구조가 예상과 다름)

In [11]:
pose = pyrosetta.pose_from_pdb(INPUT_PDB)
print("num_chains:", pose.num_chains())

def find_peptide_chain_pose(pose, peptide_len=14):
    info = []
    for ch in range(1, pose.num_chains()+1):
        seq = pose.chain_sequence(ch)
        info.append((ch, len(seq), seq))
    df = pd.DataFrame(info, columns=["pose_chain_id", "length", "sequence"])
    display(df)
    hits = [ch for ch, ln, seq in info if ln == peptide_len]
    if len(hits) != 1:
        raise RuntimeError(f"길이=={peptide_len} 체인 탐지 결과가 1개가 아닙니다: {hits}")
    return hits[0]

peptide_chain_id = find_peptide_chain_pose(pose, peptide_len=14)
print("peptide_chain_id:", peptide_chain_id)

num_chains: 2


,pose_chain_id,length,sequence
0,1,14,AGCKNFFWKTFTSC
1,2,369,MDMADEPLNGSHTWLSIPFDLNGSVVSTNTSNQTEPYYDLTSNAVLTFIYFVVCIIGLCGNTLVIYVILRYAKMKTITNIYILNLAIADELFMLGLPFLAMQVALVHWPFGKAICRVVMTVDGINQFTSIFCLTVMSIDRYLAVVHPIKSAKWRRPRTAKMITMAVWGVSLLVILPIMIYAGLRSNQWGRSSCTIN...


peptide_chain_id: 1


# 4. 체인 표준화: A=리셉터, B=펩타이드로 강제 저장

여기가 아키텍처 핵심 결함(1) 해결 포인트입니다.

입력 순서/변환 과정에서 체인 A/B가 바뀌어도, 이 단계에서:
- 리셉터(=펩타이드가 아닌 체인들)을 먼저 붙이고
- 펩타이드를 뒤에 붙여

`standardized_raw.pdb`를 항상 **A=리셉터, B=펩타이드**로 만듭니다.

In [12]:
def extract_chain_pose_by_dump(original_pose, chain_id: int):
    """단순/설명 가능한 방식: 전체 dump 후 특정 체인만 필터링해서 다시 로드"""
    tmp_full = "__tmp_full.pdb"
    tmp_chain = f"__tmp_chain_{chain_id}.pdb"
    original_pose.dump_pdb(tmp_full)

    first_res = original_pose.chain_begin(chain_id)
    pdbinfo = original_pose.pdb_info()
    chain_letter = pdbinfo.chain(first_res) if pdbinfo is not None else ""
    if not chain_letter or chain_letter.strip() == "":
        chain_letter = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"[chain_id-1]

    with open(tmp_full, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    with open(tmp_chain, "w", encoding="utf-8") as out:
        for line in lines:
            if (line.startswith("ATOM") or line.startswith("HETATM")) and len(line) > 21:
                if line[21] == chain_letter:
                    out.write(line)
            if line.startswith("TER"):
                out.write(line)
        out.write("END\n")

    new_pose = pyrosetta.pose_from_pdb(tmp_chain)
    os.remove(tmp_full)
    os.remove(tmp_chain)
    return new_pose

def standardize_to_AB(pose, peptide_chain_id, out_pdb="standardized_raw.pdb"):
    receptor_chains = [ch for ch in range(1, pose.num_chains()+1) if ch != peptide_chain_id]
    if not receptor_chains:
        raise RuntimeError("receptor chain이 없습니다.")

    rec_pose = extract_chain_pose_by_dump(pose, receptor_chains[0])
    for ch in receptor_chains[1:]:
        rec_pose.append_pose_by_jump(extract_chain_pose_by_dump(pose, ch), rec_pose.total_residue())

    pep_pose = extract_chain_pose_by_dump(pose, peptide_chain_id)
    rec_pose.append_pose_by_jump(pep_pose, rec_pose.total_residue())

    rec_pose.dump_pdb(out_pdb)
    print(f"[OK] standardized saved -> {out_pdb} (A=receptor, B=peptide)")
    return rec_pose

standard_pose = standardize_to_AB(pose, peptide_chain_id, out_pdb="standardized_raw.pdb")

[OK] standardized saved -> standardized_raw.pdb (A=receptor, B=peptide)


# 5. 표준화 자동 검증(PASS/FAIL) + 체인/잔기 범위 출력

- 체인 A 존재
- 체인 B 존재
- 체인 B 길이==14
- 체인 A 길이>14

이 조건을 만족하면 “표준화 성공”으로 판단합니다.

In [13]:
df_std = summarize_pdb("standardized_raw.pdb", show_seq="head")
display(df_std)

rowA = df_std[df_std["chain"] == "A"]
rowB = df_std[df_std["chain"] == "B"]

assert len(rowA) == 1, "체인 A가 없습니다"
assert len(rowB) == 1, "체인 B가 없습니다"
assert int(rowB["length"].iloc[0]) == 14, f"체인 B 길이가 14가 아닙니다: {rowB['length'].iloc[0]}"
assert int(rowA["length"].iloc[0]) > 14, f"체인 A 길이가 너무 짧습니다: {rowA['length'].iloc[0]}"

print("[PASS] standardized_raw.pdb: A=receptor, B=peptide(len=14)")

,chain,length,pdb_res_min,pdb_res_max,ranges,seq
0,A,369,1,369,1-369,MDMADEPLNGSHTWLSIPFDLNGSVVSTNTSNQTEPYYDLTSNAVLTFIYFVVCIIGLCG...
1,B,14,370,383,370-383,AGCKNFFWKTFTSC


[PASS] standardized_raw.pdb: A=receptor, B=peptide(len=14)


In [14]:
# =========================================================
# [3D VIEW] 표준화 결과: A=Receptor, B=Peptide 3D 확인
# - receptor(Chain A): lightblue cartoon + 반투명 surface
# - peptide(Chain B): orange cartoon + stick
# - 마우스로 회전/줌하여 인터페이스 확인 가능
# =========================================================

print("🧪 표준화된 구조 3D 뷰 (A=receptor lightblue, B=peptide orange)")
print("   마우스 드래그: 회전 | 스크롤: 줌 | 우클릭 드래그: 이동\n")

show_structure_3d("standardized_raw.pdb",
                  surface_receptor=True,
                  label="Standardized: A=Receptor, B=Peptide(SST14)")

🧪 표준화된 구조 3D 뷰 (A=receptor lightblue, B=peptide orange)
   마우스 드래그: 회전 | 스크롤: 줌 | 우클릭 드래그: 이동



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 6. peptide-only Relax (리셉터 고정)

표준화 후에는:
- Chain A(=pose chain 1) = 리셉터
- Chain B(=pose chain 2) = 펩타이드

따라서 **펩타이드 체인(2)만 MoveMap을 풀어서** FastRelax를 수행합니다.

In [15]:
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.core.select.residue_selector import ChainSelector

def relax_peptide_only(in_pdb="standardized_raw.pdb", out_pdb="standardized_relaxed.pdb", peptide_chain_number=2):
    pose = pyrosetta.pose_from_pdb(in_pdb)

    mm = MoveMap()
    mm.set_bb(False); mm.set_chi(False); mm.set_jump(False)

    pep_selector = ChainSelector(peptide_chain_number)
    pep_res = pep_selector.apply(pose)
    for i in range(1, pose.total_residue()+1):
        if pep_res[i]:
            mm.set_bb(i, True)
            mm.set_chi(i, True)

    scorefxn = pyrosetta.get_score_function()
    relax = FastRelax()
    relax.set_scorefxn(scorefxn)
    relax.set_movemap(mm)

    pre = scorefxn(pose)

    # --- 진행 상태 표시 ---
    print(f"⏳ Peptide-only FastRelax 시작... (repeats=3, 수 분 소요될 수 있음)")
    print(f"   score BEFORE relax: {pre:.2f}")
    t0 = time.time()

    pbar = tqdm(total=1, desc="FastRelax (peptide only)", bar_format="{l_bar}{bar}| {elapsed}<{remaining}")
    relax.apply(pose)
    pbar.update(1)
    pbar.close()

    elapsed = time.time() - t0
    post = scorefxn(pose)
    # --- 결과 ---

    pose.dump_pdb(out_pdb)
    print(f"✅ Relax 완료 ({elapsed:.1f}s) -> {out_pdb}")
    print(f"   score AFTER relax : {post:.2f}  (Δ = {post - pre:+.2f})")
    return pre, post

pre_score, post_score = relax_peptide_only("standardized_raw.pdb", "standardized_relaxed.pdb", peptide_chain_number=2)

⏳ Peptide-only FastRelax 시작... (repeats=3, 수 분 소요될 수 있음)
   score BEFORE relax: 49.00


FastRelax (peptide only):   0%|          | 00:00<?

✅ Relax 완료 (12.6s) -> standardized_relaxed.pdb
   score AFTER relax : -299.56  (Δ = -348.55)


In [16]:
# =========================================================
# [3D VIEW] Peptide Relax 전후 비교
# - 왼쪽: standardized_raw.pdb (Relax 전)
# - 오른쪽: standardized_relaxed.pdb (Relax 후)
# - 펩타이드 구조 변화를 나란히 확인
# =========================================================

print("🔄 Relax 전후 펩타이드 구조 비교 (마우스 회전/줌 가능)")
print(f"   score 변화: {pre_score:.2f} → {post_score:.2f} (Δ = {post_score - pre_score:+.2f})\n")

show_comparison_3d(
    ["standardized_raw.pdb", "standardized_relaxed.pdb"],
    labels=[f"Before Relax\nscore={pre_score:.1f}", f"After Relax\nscore={post_score:.1f}"],
    width=900, height=450
)

🔄 Relax 전후 펩타이드 구조 비교 (마우스 회전/줌 가능)
   score 변화: 49.00 → -299.56 (Δ = -348.55)



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 7. stability/PK proxy 점수(데모용)

혈중 안정성(6~10일)을 정확히 예측하는 것은 별도 모델/실험이 필요하지만,
데모 아키텍처 관점에서는 **서열 기반 proxy 점수 레이어가 존재**하는 것이 중요합니다.

- cleavage_risk: K/R, 방향족(F/Y/W) 기반 가중
- pk_penalty: 소수성 과다 + 전하 과다 페널티(간이)

※ 실제 연구 단계에서는 PeptideCutter 같은 도구/모델을 붙여 고도화 가능

In [17]:
HYDROPHOBIC = set(list("AILMFWVY"))
BASIC = set(list("KRH"))
ACIDIC = set(list("DE"))

def stability_pk_proxy_scores(seq: str):
    seq = seq.strip().upper()
    kr = sum(1 for x in seq if x in "KR")
    arom = sum(1 for x in seq if x in "FYW")
    cleavage_risk = 2.0 * kr + 1.0 * arom

    hyd = sum(1 for x in seq if x in HYDROPHOBIC)
    hydrophobic_fraction = hyd / max(len(seq), 1)

    pos = sum(1 for x in seq if x in BASIC)
    neg = sum(1 for x in seq if x in ACIDIC)
    net_charge_proxy = pos - neg

    pk_penalty = 5.0 * max(0.0, hydrophobic_fraction - 0.50) + 0.5 * abs(net_charge_proxy)
    return {
        "cleavage_risk": cleavage_risk,
        "hydrophobic_fraction": hydrophobic_fraction,
        "net_charge_proxy": net_charge_proxy,
        "pk_penalty": pk_penalty,
    }

orig_pose = pyrosetta.pose_from_pdb("standardized_relaxed.pdb")
orig_seq = orig_pose.chain_sequence(2)
print("Original peptide seq:", orig_seq)
stability_pk_proxy_scores(orig_seq)

Original peptide seq: AGCKNFFWKTFTSC


{'cleavage_risk': 8.0,
 'hydrophobic_fraction': 0.35714285714285715,
 'net_charge_proxy': 2,
 'pk_penalty': 1.0}

# 8. Interface 분석(dG/dSASA)

InterfaceAnalyzerMover는 인터페이스 정의 옵션을 갖고 있으며(서로 배타),
우리는 **체인 표준화(A/B)**를 통해 “무엇을 측정하는 dG인지”가 흔들리지 않게 합니다.
(참고 문서: InterfaceAnalyzerMover 파라미터/상호배타 옵션 설명)
- https://docs.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/analysis/InterfaceAnalyzerMover

In [18]:
from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover

def analyze_interface(pose):
    # 표준화 후에는 사실상 receptor-peptide 인터페이스만 보도록 구조를 단순화했다는 점이 핵심
    iam = InterfaceAnalyzerMover(1)
    iam.set_pack_separated(True)
    iam.set_compute_packstat(True)
    iam.apply(pose)
    return iam.get_interface_dG(), iam.get_interface_delta_sasa()

test_pose = pyrosetta.pose_from_pdb("standardized_relaxed.pdb")
dG0, dSASA0 = analyze_interface(test_pose)
print(f"baseline dG={dG0:.2f} REU, dSASA={dSASA0:.2f}")

baseline dG=-41.05 REU, dSASA=2062.87


# 9. FastDesign 후보 20개 생성 + DataFrame으로 즉시 확인

- 수용체(리셉터) 고정
- 펩타이드 Cys(고리 유지) 고정
- 후보 20개 생성
- 각 후보에 대해
  - 서열
  - dG/dSASA
  - stability proxy
  - 허용 design 포지션 밖 변이 발생 여부
를 테이블로 바로 출력합니다.

In [ ]:
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task.operation import OperateOnResidueSubset, PreventRepackingRLT
from pyrosetta.rosetta.core.select.residue_selector import (
    ChainSelector, NotResidueSelector, ResidueNameSelector, AndResidueSelector, OrResidueSelector
)
from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign

def peptide_seq(pose, peptide_chain_id=2):
    return pose.chain_sequence(peptide_chain_id)

def diff_positions(original, new):
    return [i for i, (o, n) in enumerate(zip(original, new), start=1) if o != n]

def build_task_factory_lock_receptor_and_cys(pose, peptide_chain_id=2):
    pep_selector = ChainSelector(peptide_chain_id)
    rec_selector = NotResidueSelector(pep_selector)

    cys_selector = ResidueNameSelector("CYS")
    pep_cys_selector = AndResidueSelector(pep_selector, cys_selector)

    cant_touch = OrResidueSelector(rec_selector, pep_cys_selector)

    tf = TaskFactory()
    tf.push_back(OperateOnResidueSubset(PreventRepackingRLT(), cant_touch))
    return tf

def fastdesign_candidates(input_pdb="standardized_relaxed.pdb", n=3, design_pos="2,3,6,7,8,10"):
    design_positions = [int(x.strip()) for x in design_pos.split(",") if x.strip()]
    allowed = set(design_positions)

    base_pose = pyrosetta.pose_from_pdb(input_pdb)
    orig_seq = peptide_seq(base_pose, 2)

    rows = []
    os.makedirs("candidates", exist_ok=True)

    t_total = time.time()
    timings = []

    pbar = tqdm(range(1, n + 1), desc="FastDesign 후보 생성", unit="candidate",
                bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]")

    for k in pbar:
        t_start = time.time()
        pose = pyrosetta.pose_from_pdb(input_pdb)

        tf = build_task_factory_lock_receptor_and_cys(pose, peptide_chain_id=2)
        fd = FastDesign()
        fd.set_scorefxn(pyrosetta.get_score_function())
        fd.set_task_factory(tf)
        fd.apply(pose)

        new_seq = peptide_seq(pose, 2)
        diffs = diff_positions(orig_seq, new_seq)
        outside = [p for p in diffs if (p not in allowed and orig_seq[p - 1] != "C")]

        dG, dSASA = analyze_interface(pose)
        stab = stability_pk_proxy_scores(new_seq)

        out_name = f"candidate_{k:03d}.pdb"
        out_path = os.path.join("candidates", out_name)
        pose.dump_pdb(out_path)

        elapsed_k = time.time() - t_start
        timings.append(elapsed_k)
        avg_time = sum(timings) / len(timings)
        remaining_est = avg_time * (n - k)

        pbar.set_postfix_str(
            f"#{k} {new_seq[:8]}… dG={dG:.1f} | 이번={elapsed_k:.0f}s 평균={avg_time:.0f}s 남은≈{remaining_est/60:.1f}m"
        )

        rows.append({
            "candidate": out_name,
            "pdb_path": out_path,
            "seq": new_seq,
            "dG_REU": dG,
            "dSASA": dSASA,
            "mut_positions": diffs,
            "mut_outside_allowed": outside,
            "design_time_s": elapsed_k,
            **stab
        })

    total_elapsed = time.time() - t_total
    print(f"\n✅ FastDesign 완료: {n}개 후보, 총 {total_elapsed/60:.1f}분 (평균 {total_elapsed/n:.0f}s/후보)")

    df = pd.DataFrame(rows)
    df["rank_score"] = (-df["dG_REU"]) - 0.5 * df["cleavage_risk"] - 1.0 * df["pk_penalty"]
    df = df.sort_values(["rank_score"], ascending=False).reset_index(drop=True)
    return df, orig_seq, design_positions

df_candidates, original_pep, design_positions = fastdesign_candidates(n=20, design_pos="2,3,6,7,8,10")
print("Original peptide:", original_pep)
print("Design positions:", design_positions)
df_candidates.head(10)

FastDesign 후보 생성:   0%|          | 0/20 [00:00<?, ?candidate/s]

# 9-1. mut_outside_allowed 후보 자동 제거(필터링)

데모 스토리 포인트:
- FastDesign이 만든 후보 중 “허용한 설계 포지션 밖 변이”가 생긴 후보는
  **정책 위반**으로 자동 탈락시키고,
- 이후 FlexPepDock / PyMOL 스냅샷은 “정책 준수 후보”만 대상으로 진행합니다.

In [ ]:
import pandas as pd

assert "df_candidates" in globals(), "df_candidates가 없습니다. FastDesign 후보 생성 셀을 먼저 실행하세요."
df = df_candidates.copy()

assert "mut_outside_allowed" in df.columns, "mut_outside_allowed 컬럼이 없습니다."

def is_violation(x):
    if isinstance(x, bool):
        return bool(x)
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return False
    if isinstance(x, (str, list, tuple, set, dict)):
        return len(x) > 0
    return True

df["__violation__"] = df["mut_outside_allowed"].apply(is_violation)

df_pass = df[df["__violation__"] == False].drop(columns=["__violation__"]).reset_index(drop=True)
df_fail = df[df["__violation__"] == True].drop(columns=["__violation__"]).reset_index(drop=True)

print(f"[필터링 결과] 전체 후보: {len(df)}")
print(f"  - 통과(정책 준수): {len(df_pass)}")
print(f"  - 탈락(mut_outside_allowed): {len(df_fail)}")

display(df_pass.head(10))

df_candidates_filtered = df_pass

In [ ]:
# =========================================================
# [3D VIEW] FastDesign 필터링 통과 Top3 후보 3D 비교
# - 정책 준수(mut_outside_allowed 없음) 후보만 대상
# - rank_score 상위 3개를 나란히 비교
# =========================================================

_topk_fd = min(3, len(df_candidates_filtered))
_fd_top = df_candidates_filtered.sort_values("rank_score", ascending=False).head(_topk_fd)

_fd_paths = _fd_top["pdb_path"].tolist()
_fd_labels = [
    f"#{i+1} {row['candidate']}\nseq={row['seq'][:10]}…\ndG={row['dG_REU']:.1f}"
    for i, row in _fd_top.iterrows()
]

print(f"🧬 FastDesign 필터링 통과 Top{_topk_fd} 후보 3D 비교")
print("   receptor=lightblue, peptide=orange (마우스 회전/줌 가능)\n")

if _topk_fd >= 2:
    show_comparison_3d(_fd_paths, labels=_fd_labels, width=900, height=400)
else:
    show_structure_3d(_fd_paths[0], label=_fd_labels[0])

# 10. FlexPepDock로 상위 후보 refine + 재스코어링

FastDesign이 만든 후보는 “생성” 단계,
FlexPepDock은 “정밀 검증/리파인” 단계입니다.

여기서는 후보 상위 TopK(예: 10개)를 refine하고,
refine 후 dG/dSASA가 어떻게 변하는지 확인합니다.

※ 데모에서 더 깔끔하게 하려면, **필터링 통과 후보(df_candidates_filtered)** 에서 TopK를 뽑아 refine 하세요.

In [ ]:
from pyrosetta.rosetta.protocols.flexpep_docking import FlexPepDockingProtocol

def flexpepdock_refine(df_in, topk=10):
    os.makedirs("refined", exist_ok=True)
    rows = []
    top = df_in.head(topk)
    n_total = len(top)

    t_total = time.time()
    timings = []

    pbar = tqdm(top.iterrows(), total=n_total, desc="FlexPepDock Refine", unit="candidate",
                bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]")

    for idx, (i, row) in enumerate(pbar, start=1):
        t_start = time.time()

        in_pdb = row["pdb_path"]
        pose = pyrosetta.pose_from_pdb(in_pdb)

        fpd = FlexPepDockingProtocol()
        fpd.apply(pose)

        dG, dSASA = analyze_interface(pose)
        seq = peptide_seq(pose, 2)
        stab = stability_pk_proxy_scores(seq)

        out_name = os.path.basename(row["candidate"]).replace("candidate_", "refined_")
        out_path = os.path.join("refined", out_name)
        pose.dump_pdb(out_path)

        elapsed_k = time.time() - t_start
        timings.append(elapsed_k)
        avg_time = sum(timings) / len(timings)
        remaining_est = avg_time * (n_total - idx)

        pbar.set_postfix_str(
            f"{row['candidate']} dG={dG:.1f} | {elapsed_k:.0f}s 남은≈{remaining_est/60:.1f}m"
        )

        rows.append({
            "input": row["candidate"],
            "input_pdb": in_pdb,
            "output": out_name,
            "pdb_path": out_path,
            "seq": seq,
            "dG_REU": dG,
            "dSASA": dSASA,
            **stab,
            "mut_outside_allowed": row.get("mut_outside_allowed", None),
            "refine_time_s": elapsed_k,
        })

    total_elapsed = time.time() - t_total
    print(f"\n✅ FlexPepDock 완료: {n_total}개 후보, 총 {total_elapsed/60:.1f}분 (평균 {total_elapsed/n_total:.0f}s/후보)")

    df = pd.DataFrame(rows)
    df["rank_score"] = (-df["dG_REU"]) - 0.5 * df["cleavage_risk"] - 1.0 * df["pk_penalty"]
    df = df.sort_values(["rank_score"], ascending=False).reset_index(drop=True)
    return df

# 필터링 통과 후보로 refine 권장
df_refined = flexpepdock_refine(df_candidates_filtered, topk=min(10, len(df_candidates_filtered)))
df_refined

In [ ]:
# =========================================================
# [3D VIEW] FlexPepDock Refined Top3 후보 인터랙티브 3D 비교
# - 리셉터 표면 + 펩타이드 cartoon/stick
# - 데모 클라이맥스: 최종 후보 구조를 직접 회전/줌하며 확인
# =========================================================

_topk_refined = min(3, len(df_refined))
_refined_top = df_refined.sort_values("rank_score", ascending=False).head(_topk_refined)

_refined_paths = _refined_top["pdb_path"].tolist()
_refined_labels = [
    f"#{i+1} {row['output']}\ndG={row['dG_REU']:.1f} dSASA={row['dSASA']:.0f}"
    for i, row in _refined_top.iterrows()
]

print(f"🔬 Refined Top{_topk_refined} 후보 3D 비교 (마우스로 회전/줌 가능)")
print("   receptor=lightblue (surface), peptide=orange (cartoon+stick)\n")

# 개별 뷰: 첫 번째 후보를 표면 렌더링으로 크게 표시
show_structure_3d(_refined_paths[0], width=800, height=500,
                  surface_receptor=True,
                  label=f"Best: {_refined_top.iloc[0]['output']}")

In [ ]:
# Top3 나란히 그리드 비교
if _topk_refined > 1:
    show_comparison_3d(_refined_paths, labels=_refined_labels, width=900, height=400)
else:
    print("후보가 1개뿐이므로 그리드 비교를 건너뜁니다.")

# 10-1. Top3 후보 PyMOL 스냅샷 자동 생성

전제:
- 로컬에 PyMOL이 설치되어 있어야 합니다.
- 커맨드라인에서 `pymol` 명령이 실행 가능해야 합니다.
  - 예: `pymol -cq script.pml`

동작:
- (기본) df_refined에서 rank_score 기준 Top3를 선택
- 각 Top 후보 PDB를 로드하고
  - chain A: receptor (lightblue)
  - chain B: peptide (orange)
- PNG 스냅샷을 `pymol_snapshots/`에 저장합니다.

In [ ]:
# =========================================================
# [PyMOL SNAPSHOT] Top3 후보 자동 스냅샷 생성
# - df_refined (또는 df_candidates_filtered)에서 rank_score 기준 Top3
# - chain A: receptor (lightblue), chain B: peptide (orange)
# =========================================================

from pathlib import Path

# --- PyMOL 초기화 (pymol2 API 우선, fallback으로 pymol.cmd) ---
_pymol_session = None
_pymol_cmd = None

try:
    import pymol2
    _pymol_session = pymol2.PyMOL()
    _pymol_session.start()
    _pymol_cmd = _pymol_session.cmd
    _pymol_mode = "pymol2"
    print("[OK] PyMOL initialized via pymol2 API (headless)")
except Exception:
    try:
        import pymol
        pymol.finish_launching(["pymol", "-cq"])  # headless, quiet
        from pymol import cmd as _pymol_cmd
        _pymol_mode = "pymol_cmd"
        print("[OK] PyMOL initialized via pymol.cmd (headless)")
    except Exception as e:
        _pymol_cmd = None
        _pymol_mode = "unavailable"
        print(f"[WARN] PyMOL 사용 불가 - 스냅샷을 건너뜁니다: {e}")

# --- 설정 ---
DF_NAME = "df_refined"
PDB_COL = "pdb_path"
SCORE_COL = "rank_score"
TOPK = 3

OUTDIR = Path("pymol_snapshots")
OUTDIR.mkdir(exist_ok=True)

if _pymol_cmd is not None:
    assert DF_NAME in globals(), f"{DF_NAME} 가(이) 없습니다. FlexPepDock refine 셀을 먼저 실행하세요."
    df = globals()[DF_NAME].copy()

    assert PDB_COL in df.columns, f"{DF_NAME}에 {PDB_COL} 컬럼이 없습니다."
    assert SCORE_COL in df.columns, f"{DF_NAME}에 {SCORE_COL} 컬럼이 없습니다."

    df_top = df.sort_values(SCORE_COL, ascending=False).head(TOPK).reset_index(drop=True)

    print(f"\n[Top{TOPK} 후보] (정렬 기준: {SCORE_COL})")
    display(df_top[[PDB_COL, SCORE_COL] + [c for c in ["seq", "dG_REU", "dSASA"] if c in df_top.columns]])

    def render_snapshot(pdb_path: str, out_png: str,
                        width=1600, height=1200, dpi=300,
                        chainA_color="lightblue", chainB_color="orange"):
        """PyMOL cmd를 이용해 headless 렌더링 후 PNG 저장."""
        cmd = _pymol_cmd
        cmd.reinitialize()
        cmd.load(pdb_path, "complex")

        cmd.hide("everything", "all")
        cmd.show("cartoon", "all")

        cmd.color(chainA_color, "chain A")
        cmd.color(chainB_color, "chain B")

        cmd.bg_color("white")
        try:
            cmd.set("ray_opaque_background", 0)
        except Exception:
            pass

        cmd.orient("all")
        cmd.zoom("all")

        cmd.png(out_png, width=width, height=height, dpi=dpi, ray=1)
        return out_png

    # 실행
    created = []
    for i, row in tqdm(df_top.iterrows(), total=len(df_top), desc="PyMOL 스냅샷 렌더링", unit="img"):
        pdb_path = str(row[PDB_COL])
        out_png = str(OUTDIR / f"top{i+1}_snapshot.png")
        print(f"  [Render] {pdb_path} -> {out_png}")
        render_snapshot(pdb_path, out_png)
        created.append(out_png)

    print(f"\n✅ 스냅샷 {len(created)}개 생성 완료:")
    for p in created:
        print(f"  - {p}")

    # pymol2 세션 정리(리소스 해제)
    if _pymol_mode == "pymol2" and _pymol_session is not None:
        _pymol_session.stop()
        print("\n[OK] pymol2 session stopped")
else:
    print("[SKIP] PyMOL 미설치 → 스냅샷 생성 건너뜀. 수동으로 PyMOL에서 확인하세요.")

# 11. Relax 이후에도 체인 표준화 유지 확인

표준화된 체인이 Relax 이후에도 유지되는지 요약표로 다시 확인합니다.

In [ ]:
summarize_pdb("standardized_relaxed.pdb", show_seq="head")

# 12. 정리

이 노트북 데모에서 보여준 것:

1) 체인 A/B가 입력 순서에 따라 뒤바뀔 수 있는 위험을 **표준화 단계**로 제거  
2) 펩타이드만 Relax하여 리셉터 변형 치팅을 줄임  
3) FastDesign으로 후보 20개를 만들고, 테이블로 스코어/서열을 즉시 확인  
4) `mut_outside_allowed`로 **정책 위반 후보 자동 탈락**  
5) FlexPepDock으로 상위 후보를 refine하여 “검증 단계”를 구현  
6) Top3 후보를 PyMOL로 스냅샷 자동 생성하여 시각적으로 결과를 제시  
7) 혈중 안정성 목표에 대응하는 최소 stability/PK proxy 레이어 포함  

추후 확장:
- SSTR1/3/5 오프타깃에도 동일 후보를 넣어 Selectivity score 계산  
- stability/PK proxy를 실제 도구/모델/실험 기반으로 고도화